# Hybrid Model using game tags

First only content based filtering

In [1]:
#get the data 
csvPath = "../Datasets/dataWithRatings.csv"

In [4]:
import pandas as pd
df = pd.read_csv (csvPath)

# Generate Group

In [5]:
users_ratings = df.groupby(['user_id']).count()
selected = users_ratings['Rating M1'] > 100
selected_users = users_ratings.loc[selected]
random_selected = selected_users.sample(n=5) # sample() returns now n random rows from the dataframe. The returned object is a dataframe with five rows. 
select_column_df = random_selected.reset_index()['user_id'] # reset_index() create a new index, and the userId became a column. Then, we can filter using the column name
group_users = list(select_column_df) # iloc select by index, since our dataframe only has one row we read it from the index 0
print(group_users)

[49893565, 22301321, 48798067, 47457723, 138941587]


Find seen seen and unseen games

In [23]:
group_ratings = df.loc[df['user_id'].isin(group_users)]
# print(group_ratings.head(10))
# display(group_ratings.head(10))
all_games = set(df.index.tolist())
num_ratings_df = df.groupby(['name']).count()
considered_games = set(num_ratings_df.loc[num_ratings_df['user_id'] > 10].reset_index()['name'])

group_seen_games = set(group_ratings['name'].tolist())
group_unseen_games = considered_games - group_seen_games

print(len(all_games))
print(len(considered_games))
print(len(group_seen_games))
print(len(group_unseen_games))

36257
422
441
172


In [71]:
rated_games_dfzz = pd.DataFrame()
for game in group_seen_games:
  rated_games_dfzz=rated_games_dfzz.append(group_ratings.loc[group_ratings["name"] == game])


unrated_games_dfzz = get_unrated_games_user(rated_games_dfzz)


rated_games_dfzz = rated_games_dfzz[['user_id','name', 'game_description', 'Rating M1', 'Rating M2']]
unrated_games_dfzz = unrated_games_dfzz[['name', 'game_description']]

In [76]:
display(rated_games_dfzz)

,user_id,name,game_description,Rating M1,Rating M2
34393,138941587,Humanity Asset,About This Game Humanity Asset is a Metroid-s...,2,5
16260,138941587,Out There Somewhere,About This Game From the artists behind the s...,2,5
22379,49893565,Worms Reloaded,About This Game Ten years on from Worms™ Arma...,1,4
22409,47457723,Worms Reloaded,About This Game Ten years on from Worms™ Arma...,2,5
29749,47457723,The Guild II,About This Game The Guild 2 is the shining su...,3,5
...,...,...,...,...,...
35252,49893565,Cortex Command,About This Game A project over eleven years i...,1,2
35257,47457723,Cortex Command,About This Game A project over eleven years i...,1,5
23660,22301321,The Misadventures of P.B. Winterbottom,About This Game Create your own paradox… for ...,1,2
35921,47457723,Outcast 1.1,About This Game Fresh3D is bringing you Outca...,1,1


### Rated games by the user

In [49]:
def get_rated_games_user(selected_user):
    selected_user_ratings = df.loc[df['user_id'] == selected_user]
    # selected_user_ratings = selected_user_ratings.sort_values(by='hours', ascending=True)
#     print("Rated games: " + str(selected_user_ratings.shape[0]))
    # display(selected_user_ratings.head(10))
    rated_games_df = selected_user_ratings

    rated_games_df = rated_games_df[['name', 'game_description', 'Rating M1', 'Rating M2']]

#     display(rated_games_df.head(10))    
    return rated_games_df


### Unrated games by the user

In [48]:
def get_unrated_games_user(rated_games_df):
    diff = set(df.index) - set(rated_games_df.index)
    unrated_games_df = df.loc[diff]
    unrated_games_df = unrated_games_df[['name', 'game_description']]
    unrated_games_df = unrated_games_df.drop_duplicates(subset ="name") #Had to remove duplicates 
#     print("Unrated games: " + .str(unrated_games_df.shape[0]))
#     display(unrated_games_df.head(10))
    return unrated_games_df

In [13]:
display(random_selected)

,name,hours,desc_snippet,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price,Rating M1,Rating M2
user_id,,,,,,,,,,,,,,,,,,,,
49893565,136,136,136,134,132,135,131,135,136,136,97,135,135,13,66,66,134,65,136,136
22301321,106,106,106,105,102,106,103,105,106,106,77,105,105,9,55,55,106,54,106,106
48798067,105,105,105,105,101,104,101,105,104,105,71,104,105,8,73,73,105,50,105,105
47457723,139,139,139,138,138,138,136,138,139,139,82,138,138,9,75,75,139,76,139,139
138941587,153,153,153,152,151,152,150,153,153,153,115,153,153,7,74,74,152,69,153,153


## Content Based Filtering


#### Train KNN for each user using Rating M1 and Rating M2 seperately for all users

In [83]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'
# initialise the vectorizer
group_matrix = np.zeros(shape=(5,1474))
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
# vectorizer = CountVectorizer()
full_predictions = pd.DataFrame()
# for each gamer in the selected user list (min gameplay time of 300
for i,gamer in enumerate(group_users):
    rated_games_df = rated_games_dfzz.dropna()
#     display(rated_games_df)
    unrated_games_df = unrated_games_dfzz
    # display(rated_games_df)
    X = vectorizer.fit_transform(rated_games_df.loc[rated_games_df["user_id"] == gamer]['game_description'])
    y = rated_games_df.loc[rated_games_df["user_id"] == gamer]['Rating M1']
    
    display(y) 
    indices = np.arange(X.shape[0])
    (
        X_train,
        X_test,
        y_train,
        y_test,
        indices_train,
        indices_test,
    ) = train_test_split(X, y, indices, test_size=0.3, random_state=101)
    linReg = LinearRegression().fit(X_train, y_train)
    neigh = KNeighborsRegressor(n_neighbors=10).fit(X_train, y_train)

    X_unrated = vectorizer.transform(unrated_games_df['game_description'].values.astype('U'))

    # print(X_unrated.shape)
    #predicting already existing rated values 
    y_pred1_reg = linReg.predict(X_test)
    y_pred1_neigh = neigh.predict(X_test)
    #predicting unrated games
    y_pred1_unrated_reg = linReg.predict(X_unrated)
    y_pred1_unrated_neigh = neigh.predict(X_unrated)
    
#         print(y_pred1.shape)

    # unrated_games_df['predicted_ratings_KNN M1'] = y_pred1_unrated_neigh
    group_matrix[i]=y_pred1_unrated_neigh.T
    unrated_games_df['predicted_ratings_Reg M1'] = y_pred1_unrated_reg

    tt = rated_games_df.iloc[indices_test]
    tt['y_test_KNN M1'] = y_pred1_neigh
    tt['y_test_Reg M1'] = y_pred1_reg

    y2 = rated_games_df.loc[rated_games_df["user_id"] == gamer]['Rating M2']
    indices2 = np.arange(X.shape[0])
    (
        X_train2,
        X_test2,
        y_train2,
        y_test2,
        indices_train2,
        indices_test2,
    ) = train_test_split(X, y2, indices2, test_size=0.3, random_state=101)
    linReg2 = LinearRegression().fit(X_train, y_train2)
    neigh2 = KNeighborsRegressor(n_neighbors=10).fit(X_train, y_train2)

    y_unrated2_Reg = linReg2.predict(X_unrated)
    y_pred2_Reg = linReg2.predict(X_test)

    y_unrated2_neigh = neigh2.predict(X_unrated)
    y_pred2_neigh = neigh2.predict(X_test)

    unrated_games_df['predicted_ratings_KNN M2'] = y_unrated2_neigh
    unrated_games_df['predicted_ratings_Reg M2'] = y_unrated2_Reg
    
    unrated_games_df_KNN_ordering = unrated_games_df.sort_values(by='predicted_ratings_KNN M2', ascending=False)
    unrated_games_df_Reg_ordering = unrated_games_df.sort_values(by='predicted_ratings_Reg M2', ascending=False)
#     display(unrated_games_df_KNN_ordering.head(10))

    tt['y_test_KNN M2'] = y_pred2_neigh
    tt['y_test_Reg M2'] = y_pred2_Reg

    full_predictions = full_predictions.append(tt)

    # except:
    #     continue

22379    1
21245    1
14191    1
17297    2
34617    1
        ..
35024    1
11324    3
30453    1
23687    1
35252    1
Name: Rating M1, Length: 135, dtype: int64

34260    1
33136    1
14207    1
11176    5
35679    1
        ..
28332    5
11370    2
34106    1
34014    2
23660    1
Name: Rating M1, Length: 105, dtype: int64

35947    1
17664    2
34267    2
35951    1
25895    1
        ..
3847     3
15778    5
30463    2
34110    3
33451    1
Name: Rating M1, Length: 105, dtype: int64

22409    2
29749    3
33250    2
36232    1
30267    1
        ..
28375    5
31829    1
34118    1
35257    1
35921    1
Name: Rating M1, Length: 138, dtype: int64

34393    2
16260    2
31123    5
21061    5
33244    2
        ..
30312    1
31470    4
35595    1
35532    3
35604    1
Name: Rating M1, Length: 153, dtype: int64

In [84]:
print(group_matrix)

[[3.4 2.6 2.1 ... 2.1 2.  1.4]
 [2.8 2.7 2.2 ... 2.6 1.7 2. ]
 [4.1 2.7 2.9 ... 3.  2.7 2.6]
 [2.5 2.  2.2 ... 1.9 1.7 2.2]
 [2.8 2.7 3.  ... 2.3 2.7 3.2]]


In [18]:
from sklearn.metrics import mean_squared_error

rmse_M1_KNN = mean_squared_error(tt['Rating M1'], tt['y_test_KNN M1'])
rmse_M2_KNN = mean_squared_error(tt['Rating M2'], tt['y_test_KNN M2'])

rmse_M1_Reg = mean_squared_error(tt['Rating M1'], tt['y_test_Reg M1'])
rmse_M2_Reg = mean_squared_error(tt['Rating M2'], tt['y_test_Reg M2'])

print('RMSE for KNN M1: ' + str(rmse_M1_KNN))
print('RMSE for KNN M2: ' + str(rmse_M2_KNN))

print('RMSE for Reg M1: ' + str(rmse_M1_Reg))
print('RMSE for Reg M2: ' + str(rmse_M2_Reg))

RMSE for KNN M1: 1.297391304347826
RMSE for KNN M2: 1.5878260869565213
RMSE for Reg M1: 1.4552917860446244
RMSE for Reg M2: 1.4242548079708306


## Collaborative Filtering 
#### UserUser with Lenskit

In [10]:
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
ratings_df1 = df[['user_id','name','Rating M1']]
ratings_df1 = ratings_df1.rename(columns={'user_id':'user', 'name': 'item', 'Rating M1': 'rating'})
display(ratings_df1)
num_recs = 10  # Number of recommmendations to generate
user_user = UserUser(15, min_nbrs=3)  # Minimum (3) and maximum (15) number of neighbors to consider
recsys = Recommender.adapt(user_user)
recsys.fit(ratings_df1)

ModuleNotFoundError: No module named 'lenskit'

In [ ]:
selected_games_useruser1 = recsys.recommend(selected_user, 10) # generate 10 recommendations for the selected user 

display(selected_games_useruser1)

,item,score
0,Farming Simulator 2011,4.828475
1,Guild Wars,4.765201
2,EVE Online,4.731073
3,Broken Sword 5 - the Serpent's Curse,4.410861
4,Fallout 4,4.238260
5,Kingdom Rush,4.149742
6,Knights of Honor,4.087136
7,Evil Genius,4.070866
8,Software Inc.,3.919878
9,DRAGON BALL XENOVERSE,3.909945


In [ ]:
temp1 = unrated_games_df_KNN_ordering[['name','predicted_ratings_KNN M1']]
temp1 = temp1.rename(columns={'name':'item'})
temp1 = pd.merge(selected_games_useruser1, temp1, on ='item') 
temp1['Weighted Score'] = 0.5*temp1['score']  +  0.5*temp1['predicted_ratings_KNN M1']
final_pred1 = temp1[['item','Weighted Score']]
display(final_pred1)


,item,Weighted Score
0,Farming Simulator 2011,3.564237
1,Guild Wars,3.732600
2,EVE Online,3.615536
3,Broken Sword 5 - the Serpent's Curse,3.805430
4,Fallout 4,3.519130
5,Kingdom Rush,3.374871
6,Knights of Honor,3.243568
7,Evil Genius,3.635433
8,Software Inc.,3.159939
9,DRAGON BALL XENOVERSE,3.354973
